In [1]:
import numpy as np
import os

os.environ["KERAS_BACKEND"]="tensorflow"

import keras

In [2]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
#scaling images
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0],"train samples")
print(x_test.shape[0],"test samples")


num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64,kernel_size=(3,3),activation ="relu"),
        keras.layers.Conv2D(64,kernel_size=(3,3),activation ="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(128, kernel_size=(3,3), activation ="relu"),
        keras.layers.Conv2D(128, kernel_size=(3,3),activation ="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes,activation = "softmax"),
    ]
)


11490434/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         147584    
                                                                 
 global_average_pooling2d (  (None, 128)               0         
 GlobalAveragePooling2D)                                

In [7]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc")
    ],
)


In [8]:
batch_size = 128

In [9]:
epochs = 20


In [10]:
callbacks=[
        keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)

score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/20
399/399 [==============================] - 287s 717ms/step - loss: 0.7597 - acc: 0.7439 - val_loss: 0.1358 - val_acc: 0.9628
Epoch 2/20
399/399 [==============================] - 279s 699ms/step - loss: 0.2177 - acc: 0.9371 - val_loss: 0.0844 - val_acc: 0.9770
Epoch 3/20
399/399 [==============================] - 279s 699ms/step - loss: 0.1513 - acc: 0.9542 - val_loss: 0.0758 - val_acc: 0.9798
Epoch 4/20
399/399 [==============================] - 286s 717ms/step - loss: 0.1241 - acc: 0.9625 - val_loss: 0.0526 - val_acc: 0.9853
Epoch 5/20
399/399 [==============================] - 278s 697ms/step - loss: 0.1022 - acc: 0.9696 - val_loss: 0.0518 - val_acc: 0.9861
Epoch 6/20
399/399 [==============================] - 287s 718ms/step - loss: 0.0901 - acc: 0.9738 - val_loss: 0.0447 - val_acc: 0.9888
Epoch 7/20
399/399 [==============================] - 288s 722ms/step - loss: 0.0815 - acc: 0.9756 - val_loss: 0.0381 - val_acc: 0.9886
Epoch 8/20
399/399 [============================

In [11]:
model.save("final_model.keras")

In [13]:
model = keras.saving.load_model("final_model.keras")

In [14]:
predictions = model.predict(x_test)

313/313 [==============================] - 15s 48ms/step
